# Black Box Data Analysis

Since the advent of IoT devices a lot of data has been available to companies in many businesses. In this project I will focus on cars' black boxes' data that is sent daily to an insurance company: these data contains information about daily distances traveled by clients, split by type of road traveled (i.e. higways, city streets...).

My main goal is to find a way of clustering clients based on the way they use their car by means of these daily distances: this type of analysis may have value in supporting the insurance company in the detection of riskier clients' behaviours and in the determination of premium amounts.

## Data Ingestion

For this analysis I have data about each black box daily distances, saved in monthly csv files.

Synce the dataset is quite large, I used Google BigQuery to store it and query it.

## Data Cleaning

The data contains daily information for each black box, but each black box can cover many years of insurance for a given client: in order to analyze the data from a common perspective I will treat each insurance annuality separately, and to do so I'm adding the annuality joining the insurance policy data.

Since the black box is installed by a third party, there is not always a correspondence between the daily data and the insurance policy coverage; this means that I have to check whether the daily distances' dates are included in the insurance annuality. Moreover, I will drop the annualities for which I have less than 100 days of data and I'll clean the ones for which I have more than 366 days of data.

All this tasks have been performed using [Google BiqQuery](https://cloud.google.com/bigquery/), here I'm reporting the queries I used. Queries results' were saved in datasets that were used in subsequent steps, the names of the datasets are indicated in the comments before the queries.

```SQL
--drop rows with dates not between start and end of the annuality -> dataset1
SELECT
	tr.*
	,po.annuality_start_date
FROM
	`k2project-201805.trips.trips` AS tr
LEFT JOIN
	`k2project-201805.trips.policies` AS po
		ON po.n_voucher = tr.n_voucher
		AND po.n_policy = tr.n_policy
WHERE
	timestamp(tr.start_date) BETWEEN timestamp(po.annuality_start_date) AND timestamp_add(timestamp(date_add(po.annuality_start_date, INTERVAL 1 YEAR)), INTERVAL 86399 SECOND)
```

```SQL
--select annualities with missing dates -> missing_days
WITH AGGR
AS
(
	SELECT
		n_voucher
		,EXTRACT(YEAR FROM annuality_start_date) AS annuality
		,MAX(start_date) AS max_date
		,MIN(start_date) AS min_date
		,COUNT(*) AS num_record
	FROM
		`k2project-201805.trips.dataset1`
	GROUP BY
		n_voucher
		,EXTRACT(YEAR FROM annuality_start_date)
)

SELECT
	n_voucher
	,annuality
	,min_date
	,max_date
	,num_record
	,DATE_DIFF(DATE(max_date), DATE(min_date), DAY) + 1 AS diff_date
FROM
	AGGR
WHERE
	num_record <> DATE_DIFF(DATE(max_date), DATE(min_date), DAY) + 1
```

```SQL
--select annualities with less than 100 days -> less100
WITH AGGR
AS
(
	SELECT
		n_voucher
		,EXTRACT(YEAR FROM annuality_start_date) AS annuality
		,COUNT(*) AS num_record
	FROM
		`k2project-201805.trips.dataset1`
	GROUP BY
		n_voucher
		,EXTRACT(YEAR FROM annuality_start_date)
)

SELECT
	n_voucher
	,annuality
	,num_record
FROM
	AGGR
WHERE
	num_record < 100
```

```SQL
--select annualities with more than 366 days -> more366
WITH AGGR
AS
(
	SELECT
		n_voucher
		,EXTRACT(YEAR FROM annuality_start_date) AS annuality
		,COUNT(*) AS num_record
	FROM
		`k2project-201805.trips.dataset1`
	GROUP BY
		n_voucher
		,EXTRACT(YEAR FROM annuality_start_date)
)

,MORE
AS
(
	SELECT
		n_voucher
		,annuality
		,num_record
	FROM
		AGGR
	WHERE
		num_record > 366
)

SELECT
	tr.n_voucher
	,EXTRACT(YEAR FROM tr.annuality_start_date) AS annuality
	,tr.n_policy
	,COUNT(*) AS num_record
FROM
	`k2project-201805.trips.dataset1` AS tr
INNER JOIN
	MORE AS M
		ON M.n_voucher = tr.n_voucher
		AND M.annuality = EXTRACT(YEAR FROM tr.annuality_start_date)
GROUP BY
	tr.n_voucher
	,EXTRACT(YEAR FROM tr.annuality_start_date)
	,tr.n_policy
```

```SQL
--drop from dataset1 annualities in missing_days, less100 and more366 -> dataset2
WITH MORE
AS
(
	SELECT
		n_voucher
		,annuality
		,n_policy
		,ROW_NUMBER() OVER (PARTITION BY n_voucher, annuality ORDER BY num_record DESC) AS Ranking
	FROM
		`k2project-201805.trips.more366`
)

SELECT
	tr.n_voucher
	,tr.n_policy
	,tr.trips_in_coverage
	,tr.start_date
	,tr.end_date
	,tr.n_trips
	,tr.km_day
	,tr.km_highway
	,tr.km_city
	,tr.km_other
	,tr.provider
	,tr.annuality_start_date
	,EXTRACT(YEAR FROM tr.annuality_start_date) AS annuality
FROM
	`k2project-201805.trips.dataset1` AS tr
LEFT JOIN
	MORE AS m
		ON m.n_voucher = tr.n_voucher
		AND m.n_policy = tr.n_policy
		AND m.annuality = EXTRACT(YEAR FROM tr.annuality_start_date)
		AND m.Ranking > 1
LEFT JOIN
	`k2project-201805.trips.less100` AS l
		ON l.n_voucher = tr.n_voucher
		AND l.n_policy = tr.n_policy
		AND l.annuality = EXTRACT(YEAR FROM tr.annuality_start_date)
LEFT JOIN
	`k2project-201805.trips.missing_days` AS d
		ON d.n_voucher = tr.n_voucher
		AND d.n_policy = tr.n_policy
		AND d.annuality = EXTRACT(YEAR FROM tr.annuality_start_date)
WHERE
	m.n_voucher IS NULL
	AND l.n_voucher IS NULL
	AND d.n_voucher IS NULL
```

```SQL
--apply all aggregations to dataset2 in order to obtain the final dataset for clustering -> dataset3
WITH QUANT
AS
(
	SELECT
		APPROX_QUANTILES(km_day, 100) AS quantiles
	FROM
		`k2project-201805.trips.dataset2`
)

SELECT
	n_voucher
	,annuality
	,STRING_AGG(bins, '' ORDER BY start_date) AS km_bin
	,STRING_AGG(quant, '' ORDER BY start_date) AS km_quant
	
	,AVG(km_day) AS km_day_mean
	,APPROX_QUANTILES(km_day, 2)[ORDINAL(2)] AS km_day_median
	,APPROX_QUANTILES(km_day, 4)[ORDINAL(2)] AS km_day_quant25
	,APPROX_QUANTILES(km_day, 4)[ORDINAL(4)] AS km_day_quant75
	,STDDEV(km_day) AS km_day_std
	,MIN(km_day) AS km_day_min
	,MAX(km_day) AS km_day_max
	,SUM(km_day) AS km_day_sum
	,COUNT(km_day) AS km_day_count
	,SUM(CASE WHEN km_day = 0 THEN 0 ELSE 1 END) AS km_day_count_zero
FROM
	(
		SELECT
			n_voucher
			,annuality
			,start_date
			,km_day
			,CASE
				WHEN km_day <=0 THEN 'A'
				WHEN km_day <= 10 THEN 'B'
				WHEN km_day <= 20 THEN 'C'
				WHEN km_day <= 30 THEN 'D'
				WHEN km_day <= 40 THEN 'E'
				WHEN km_day <= 50 THEN 'F'
				WHEN km_day <= 60 THEN 'G'
				WHEN km_day <= 100 THEN 'H'
				WHEN km_day <= 200 THEN 'I'
				WHEN km_day <= 300 THEN 'J'
				ELSE 'K'
			END AS bins
			,CASE
				WHEN tr.km_day <= q.quantiles[ORDINAL(29)] THEN 'A'
				WHEN tr.km_day <= q.quantiles[ORDINAL(50)] THEN 'B'
				WHEN tr.km_day <= q.quantiles[ORDINAL(60)] THEN 'C'
				WHEN tr.km_day <= q.quantiles[ORDINAL(70)] THEN 'D'
				WHEN tr.km_day <= q.quantiles[ORDINAL(80)] THEN 'E'
				WHEN tr.km_day <= q.quantiles[ORDINAL(90)] THEN 'F'
				WHEN tr.km_day <= q.quantiles[ORDINAL(95)] THEN 'G'
				WHEN tr.km_day <= q.quantiles[ORDINAL(99)] THEN 'H'
				ELSE 'I'
			END AS quant
		FROM
			`k2project-201805.trips.dataset2`
		CROSS JOIN
			QUANT AS q
	) AS ds
GROUP BY
	n_voucher
	,annuality
```

## Following Notebooks

- [EDA](2-EDA.ipynb)
- [Apriori](3-Apriori.ipynb)
- [Clustering on the Cloud](4a-Clustering_on_Cloud.ipynb)
- [Clustering on Premises](4b-Clustering_on_Prem.ipynb)
- [Interpreting Clusters](5-Interpreting_Clusters.ipynb)